In [12]:
import numpy as np
import os
from PIL import Image
import csv

In [13]:
train_img_path = './train_images/train_images'
test_img_path = './test_images/test_images'
train_label_file = 'train.csv'
test_label_file = 'test.csv'

In [14]:
train_img = []
train_label = []

In [15]:
for file in os.listdir(train_img_path):
    img = Image.open(train_img_path + '/' + file)
    
    train_img.append(img)

In [24]:
i = 0
with open(train_label_file, newline = '') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        if i != 0:
            train_label.append(int(row[1]))
        i = i + 1

[0,
 1,
 1,
 5,
 5,
 5,
 3,
 0,
 3,
 5,
 3,
 5,
 3,
 3,
 1,
 1,
 1,
 1,
 5,
 1,
 1,
 1,
 0,
 5,
 3,
 1,
 5,
 5,
 4,
 5,
 5,
 0,
 0,
 4,
 3,
 1,
 1,
 0,
 0,
 1,
 1,
 5,
 1,
 5,
 5,
 5,
 3,
 1,
 1,
 1,
 4,
 0,
 3,
 3,
 5,
 0,
 1,
 0,
 5,
 5,
 5,
 5,
 3,
 5,
 0,
 0,
 1,
 0,
 3,
 5,
 5,
 0,
 1,
 5,
 0,
 5,
 3,
 4,
 5,
 1,
 0,
 5,
 0,
 1,
 0,
 1,
 0,
 5,
 3,
 0,
 5,
 5,
 5,
 0,
 0,
 0,
 1,
 0,
 5,
 3,
 3,
 0,
 0,
 0,
 4,
 0,
 0,
 5,
 0,
 0,
 0,
 4,
 1,
 5,
 3,
 0,
 0,
 0,
 2,
 0,
 3,
 5,
 1,
 5,
 1,
 2,
 1,
 3,
 2,
 5,
 1,
 0,
 4,
 4,
 1,
 4,
 0,
 2,
 3,
 0,
 1,
 5,
 3,
 4,
 1,
 0,
 5,
 0,
 1,
 4,
 1,
 0,
 3,
 0,
 1,
 0,
 3,
 5,
 3,
 5,
 4,
 2,
 3,
 0,
 1,
 1,
 0,
 0,
 1,
 4,
 1,
 5,
 1,
 0,
 3,
 2,
 1,
 3,
 1,
 0,
 0,
 4,
 0,
 1,
 5,
 5,
 3,
 0,
 0,
 5,
 4,
 1,
 5,
 1,
 0,
 4,
 3,
 3,
 5,
 1,
 5,
 4,
 0,
 1,
 5,
 1,
 0,
 0,
 3,
 1,
 5,
 5,
 5,
 5,
 3,
 5,
 1,
 3,
 4,
 0,
 2,
 1,
 3,
 3,
 0,
 1,
 1,
 0,
 0,
 0,
 5,
 4,
 1,
 5,
 4,
 1,
 3,
 5,
 4,
 0,
 1,
 1,
 0,
 1,
 2,
 4,
 5,
 0,
 1,
 5,
